In [61]:
#데이터 핸들링, 기타
import numpy as np
import pandas as pd
import re
import math
import time
from datetime import datetime #JSON 날짜 변환

#시각화
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
import seaborn as sns

#크롤링 및 JSON 핸들링
from urllib.request import urlopen
from pandas.io.json import json_normalize
import requests
import bs4
import json

# 이동평균, 주가, valuation ratio

In [96]:
# fromDT = 시작 날짜
# endDT = 끝 날짜
# perPage = 페이지당 가지고 오는 데이터 개수
# term = 일간/주간/월간/분기 를 가리키는 변수
url_index_value = "http://www.wiseindex.com/DataCenter/GridData?currentPage=1&endDT=2016-10-01&fromDT=2006-01-01&index_ids=WGI0008T&isEnd=1&itemType=1&perPage=10000&term=1"
url_valuation_ratio = "http://www.wiseindex.com/DataCenter/GridData?currentPage=1&endDT=2016-10-01&fromDT=2006-01-01&index_ids=WGI0008T&isEnd=1&itemType=3&perPage=100&term=4"
#index_value는 일간, valuation_ratio는 분기별

In [97]:
#url_index_value 처리
rqt = urlopen(url_index_value) #request
jason_data = json.loads(rqt.read().decode()) #JSON 파싱 후
index_value_df = json_normalize(jason_data) #1페이지 -> to_dataframe

#url_valuation_ratio 처리
rqt = urlopen(url_valuation_ratio) #request
jason_data = json.loads(rqt.read().decode()) #JSON 파싱 후
valuation_ratio_df = json_normalize(jason_data) #1페이지 -> to_dataframe

In [98]:
#index_value DataFrame 정리
index_value_columns = list(index_value_df.columns)
index_value_columns.remove('IDX1_VAL2')
index_value_columns.remove('IDX1_VAL3')
index_value_columns.remove('ROW_IDX')
index_value_columns.remove('TOT_ROW')
index_value_df = index_value_df.reindex(columns= index_value_columns)
#column명 수정
title_index = {"IDX1_VAL1": "Index_value",'TRD_DT':"Date"}
index_value_df.rename(columns=title_index, inplace=True) # index_value_df column명 바꾸기.

#quarter_date DataFrame 정리
valuation_ratio_columns = list(valuation_ratio_df.columns)
valuation_ratio_columns.remove('ROW_IDX')
valuation_ratio_columns.remove('TOT_ROW')
valuation_ratio_df = valuation_ratio_df.reindex(columns= valuation_ratio_columns)
#column명 수정
title = {"IDX1_VAL1": "EPS","IDX1_VAL10":"BPS","IDX1_VAL2":"SPS", "IDX1_VAL3":"PER", "IDX1_VAL4":"PBR",
       "IDX1_VAL5":"PSR", "IDX1_VAL6":"EV/EBITDA", "IDX1_VAL7":"매출액growth", "IDX1_VAL8":"영업이익growth", 
         "IDX1_VAL9":"FYO growth",'TRD_DT':"Date"}
valuation_ratio_df.rename(columns=title, inplace=True) # index_value_df column명 바꾸기.

In [99]:
# reverse 하기
index_value_df = index_value_df.iloc[::-1]

In [100]:
#30일 이동평균 계산(IDX1_VAL1)
index_value = index_value_df.iloc[:, 1] 
index_MA = index_value.rolling(window=90).mean()

##기존 dataframe에 MA column 추가
index_value_df["MA"] = pd.DataFrame({"MA":index_MA}) 

In [101]:
index_value_columns_new = list(index_value_df.columns)
index_value_columns_new.remove('Index_value')
index_value_df = index_value_df.reindex(columns= index_value_columns_new)

In [102]:
#'quarter_date'기준으로 dataframe 합치기
df = pd.merge(index_value_df, valuation_ratio_df, on='Date')

In [103]:
# Date 형식 바꾸기
for i in range(0, len(df)):
    d = df['Date'].iloc[i][6:-2]
    d = int(d[:10])
    d = datetime.fromtimestamp(d).strftime('%Y-%m-%d %I:%M:%S %p')
    df['Date'].iloc[i]=d

In [108]:
df = df.iloc[::-1]

# W/L 클래스 분류

In [109]:
df.head()

,Date,MA,EPS,SPS,PER,PBR,PSR,EV/EBITDA,매출액growth,영업이익growth,FYO growth,BPS
42,2016-09-30 12:00:00 AM,2588.862444,1898.81,200485.11,311158.40,55.29,0.54,0.34,10.24,-15.82,2.35,2.93
41,2016-06-30 12:00:00 AM,2553.207333,-60.91,183663.25,317093.17,65.57,0.50,0.29,9.70,-17.67,-51.47,3.22
40,2016-03-31 12:00:00 AM,2179.086667,-1944.68,183285.52,336350.55,-840.35,0.52,0.28,9.99,-14.43,-21.33,3.07
39,2015-12-30 12:00:00 AM,2136.294889,-1944.68,183285.52,336350.55,-38.14,0.41,0.22,9.86,-14.43,-21.33,3.88
38,2015-09-30 12:00:00 AM,2442.015000,1684.07,185458.96,352887.46,46.17,0.42,0.22,8.69,-9.44,-21.33,3.71


In [110]:
df_copy = df.copy() # win_loss값을 구하기 위한 copy
df_copy['W/L'] = range(len(df_copy)) # dummy값 넣기.
df_copy.head()

,Date,MA,EPS,SPS,PER,PBR,PSR,EV/EBITDA,매출액growth,영업이익growth,FYO growth,BPS,W/L
42,2016-09-30 12:00:00 AM,2588.862444,1898.81,200485.11,311158.40,55.29,0.54,0.34,10.24,-15.82,2.35,2.93,0
41,2016-06-30 12:00:00 AM,2553.207333,-60.91,183663.25,317093.17,65.57,0.50,0.29,9.70,-17.67,-51.47,3.22,1
40,2016-03-31 12:00:00 AM,2179.086667,-1944.68,183285.52,336350.55,-840.35,0.52,0.28,9.99,-14.43,-21.33,3.07,2
39,2015-12-30 12:00:00 AM,2136.294889,-1944.68,183285.52,336350.55,-38.14,0.41,0.22,9.86,-14.43,-21.33,3.88,3
38,2015-09-30 12:00:00 AM,2442.015000,1684.07,185458.96,352887.46,46.17,0.42,0.22,8.69,-9.44,-21.33,3.71,4


In [113]:
df_copy.dropna(inplace=True)

In [122]:
df_copy = df_copy.reset_index().drop('index', 1)

In [123]:
df_copy

,Date,MA,EPS,SPS,PER,PBR,PSR,EV/EBITDA,매출액growth,영업이익growth,FYO growth,BPS,W/L
0,2016-09-30 12:00:00 AM,2588.862444,1898.81,200485.11,311158.40,55.29,0.54,0.34,10.24,-15.82,2.35,2.93,0
1,2016-06-30 12:00:00 AM,2553.207333,-60.91,183663.25,317093.17,65.57,0.50,0.29,9.70,-17.67,-51.47,3.22,1
2,2016-03-31 12:00:00 AM,2179.086667,-1944.68,183285.52,336350.55,-840.35,0.52,0.28,9.99,-14.43,-21.33,3.07,2
3,2015-12-30 12:00:00 AM,2136.294889,-1944.68,183285.52,336350.55,-38.14,0.41,0.22,9.86,-14.43,-21.33,3.88,3
4,2015-09-30 12:00:00 AM,2442.015000,1684.07,185458.96,352887.46,46.17,0.42,0.22,8.69,-9.44,-21.33,3.71,4
5,2015-06-30 12:00:00 AM,2972.705222,2196.62,186231.10,373318.93,30.33,0.54,0.28,8.51,0.84,-1.68,2.90,5
6,2015-03-31 12:00:00 AM,3280.935333,4784.91,193620.78,379926.48,23.48,0.60,0.30,9.36,9.67,41.89,2.61,6
7,2014-12-30 12:00:00 AM,3716.189556,4784.91,193620.78,379926.48,25.90,0.66,0.33,9.84,9.67,41.89,2.30,7
8,2014-09-30 12:00:00 AM,3783.974889,6192.96,193553.48,372521.77,23.88,0.78,0.41,10.49,11.10,-1.88,1.93,8
9,2014-06-30 12:00:00 AM,3566.488111,6237.21,196520.35,352182.92,26.35,0.74,0.39,10.94,11.28,19.72,2.05,9


In [124]:
# WIN_LOSS 구하기.

for i in range(len(df_copy)-1, 0, -1):
    if df_copy['MA'][i-1] > df_copy['MA'][i]:
        df_copy['W/L'].iloc[i] = '1'
    else:
        df_copy['W/L'].iloc[i] = '0'

C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [125]:
df_copy

,Date,MA,EPS,SPS,PER,PBR,PSR,EV/EBITDA,매출액growth,영업이익growth,FYO growth,BPS,W/L
0,2016-09-30 12:00:00 AM,2588.862444,1898.81,200485.11,311158.40,55.29,0.54,0.34,10.24,-15.82,2.35,2.93,0
1,2016-06-30 12:00:00 AM,2553.207333,-60.91,183663.25,317093.17,65.57,0.50,0.29,9.70,-17.67,-51.47,3.22,1
2,2016-03-31 12:00:00 AM,2179.086667,-1944.68,183285.52,336350.55,-840.35,0.52,0.28,9.99,-14.43,-21.33,3.07,1
3,2015-12-30 12:00:00 AM,2136.294889,-1944.68,183285.52,336350.55,-38.14,0.41,0.22,9.86,-14.43,-21.33,3.88,1
4,2015-09-30 12:00:00 AM,2442.015000,1684.07,185458.96,352887.46,46.17,0.42,0.22,8.69,-9.44,-21.33,3.71,0
5,2015-06-30 12:00:00 AM,2972.705222,2196.62,186231.10,373318.93,30.33,0.54,0.28,8.51,0.84,-1.68,2.90,0
6,2015-03-31 12:00:00 AM,3280.935333,4784.91,193620.78,379926.48,23.48,0.60,0.30,9.36,9.67,41.89,2.61,0
7,2014-12-30 12:00:00 AM,3716.189556,4784.91,193620.78,379926.48,25.90,0.66,0.33,9.84,9.67,41.89,2.30,0
8,2014-09-30 12:00:00 AM,3783.974889,6192.96,193553.48,372521.77,23.88,0.78,0.41,10.49,11.10,-1.88,1.93,0
9,2014-06-30 12:00:00 AM,3566.488111,6237.21,196520.35,352182.92,26.35,0.74,0.39,10.94,11.28,19.72,2.05,1


In [24]:
df_fin = df_copy.iloc[1:len(df_copy),:]

In [25]:
df_fin.to_csv("80.csv")